In [1]:
import pandas as pd
import numpy as np 
from bs4 import BeautifulSoup
import requests
import json
import urllib.request as request
import steamreviews
import re
import datetime
import time 
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import ast

## game information

In [2]:
# http://api.steampowered.com/ISteamApps/GetAppList/v0001/?key=STEAMKEY&format=json
with open('AppListV1.json', 'r') as f:
    apps = json.load(f)
    
AppList = apps['applist']['apps']['app']
len(AppList)

94290

In [3]:
def retrySession(
    retries=5,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [4]:
def webCapture(url):
    headers = {
   # pretend I am a browser
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.87 Safari/537.36',
   }
    s = requests.Session()
    response = retrySession(session=s).get(url, headers = headers, verify=False)
    soup = BeautifulSoup(response.text, 'html.parser') #parse the data
    return soup

In [ ]:
AppID = []
AppName = []
Genre = []
Tags = []
Developer = []
Publisher = []
ReviewInfor = []
ReviewStatus = []
ReleaseDate = []
IsFree = []
Price = []
count = 0

for i in range(0, len(AppList)):
    ID = str(AppList[i]["appid"])
    Name = str(AppList[i]["name"])
    url = f"https://store.steampowered.com/app/{ID}/{Name}"
    # scrapy web page
    gameDetails = webCapture(url)
    
    # get game tags
    try: 
        tag_infor = gameDetails.select("div.glance_tags.popular_tags")[0].get_text()
        tags = re.sub(r'[\t\n\r+]', ' ', tag_infor)
        tags = list(' '.join(tags.split()).split(" "))   # remove white space and put tags in a list
        genre = tags[0] # the first tag is the main genre for the game

    # get developer and publisher
        developer = re.sub(r"[\n]", '', 
                       gameDetails.select("div.dev_row > div.summary.column")[0].get_text().strip())
    
        publisher = re.sub(r"[\n]", '', 
                       gameDetails.select("div.dev_row > div.summary.column")[1].get_text().strip())

    # get "All Reviews" information 
        review_infor = re.sub(r'[\t\n\r+]', '', 
               gameDetails.select('div.user_reviews_summary_row > div.summary.column')[0].get_text())
        review_status = True if review_infor != "No user reviews" else False
        
    # get release date
        date = gameDetails.select("div.release_date > div.date")[0].get_text()
    
    except: 
        genre = None
        tags = None
        developer = None
        publisher = None
        review_infor = None
        review_status = None
        date = None
    
    # get app price
    game_link = f"http://store.steampowered.com/api/appdetails?appids={ID}"
    s = requests.Session()
    source = retrySession(session=s).get(game_link, headers={'User-Agent':'Mozilla/5.0'}, verify=False)
    json_dict = json.loads(source.text)
    
    try: 
        if json_dict[ID]["success"] == True: 
            is_free = json_dict[ID]["data"]["is_free"]
            try: 
                price = json_dict[ID]["data"]["price_overview"] if is_free == False else None
            except: 
                price = None
        else: 
            is_free = None
            price = None
    except TypeError: 
        continue
    
    AppID.append(ID)
    AppName.append(Name)
    Genre.append(genre)
    Tags.append(tags)
    Developer.append(developer)
    Publisher.append(publisher)
    ReviewInfor.append(review_infor)
    ReviewStatus.append(review_status)
    ReleaseDate.append(date)
    IsFree.append(is_free)
    Price.append(price)
    count += 1

print(count)

In [58]:
df_games = pd.DataFrame(list(zip(AppID, AppName, Genre, Tags, Developer, Publisher, ReviewInfor,
                                         ReviewStatus, ReleaseDate, IsFree, Price)),
                       columns=["AppID", "AppName", "Genre", "Tags", "Developer", "Publisher", "ReviewInfor",
                                         "ReviewStatus", "ReleaseDate", "IsFree", "Price"])
df_games.head(3)

,AppID,AppName,Genre,Tags,Developer,Publisher,ReviewInfor,ReviewStatus,ReleaseDate,IsFree,Price
0,5702,BLUR Trailer 3,None,None,None,None,None,None,None,None,None
1,5703,StargateResistance,None,None,None,None,None,None,None,False,None
2,5706,Guns Of Icarus 1,None,None,None,None,None,None,None,False,None


In [60]:
df_games.to_csv("steam_games.csv",index=False)

## clean data

In [2]:
df_games = pd.read_csv("steam_games.csv")
print(len(df_games))
df_games.head()

,AppID,AppName,Genre,Tags,Developer,Publisher,ReviewInfor,ReviewStatus,ReleaseDate,IsFree,Price
0,926170,Achromatic,Indie,"['Indie', 'Puzzle']",Studio Goya,Studio Goya,Positive(13)- 92% of the 13 user reviews for t...,True,"Sep 28, 2018",False,"{'currency': 'USD', 'initial': 499, 'final': 4..."
1,926180,Bubsy: Paws on Fire!,Action,"['Action', 'Adventure', 'Runner', 'Indie', 'Ps...",Choice Provisions,Accolade,Mostly Positive(28)- 71% of the 28 user review...,True,"May 16, 2019",False,"{'currency': 'USD', 'initial': 2499, 'final': ..."
2,926210,Incoming Evil,Indie,"['Indie', 'Action', 'First-Person', 'Horror', ...",Allyi Games,Allyi Games,9 user reviews- Need more user reviews to gene...,True,"Oct 5, 2019",False,"{'currency': 'USD', 'initial': 199, 'final': 7..."
3,926220,LA Deadzone,Action,"['Action', 'Gore', 'Violent', 'Adventure', 'VR']",Castle Steps LLC,Castle Steps LLC,No user reviews,False,"Apr 20, 2019",False,NaN
4,926240,PsyBurst,Indie,"['Indie', 'Action', 'Fighting', 'Side', 'Scrol...",Hakagame,Hakagame,2 user reviews- Need more user reviews to gene...,True,"Sep 5, 2018",False,"{'currency': 'USD', 'initial': 299, 'final': 2..."


In [9]:
# remove free games
paid_games = df_games[df_games["IsFree"]==False]

# remove paid games without reviews
print(len(paid_games[paid_games["ReviewInfor"]=="No user reviews"]))
paid_games_withreviews = paid_games[paid_games["ReviewInfor"]!="No user reviews"]

# remove nan in reviews 
paid_games_withreviews = paid_games_withreviews[paid_games_withreviews["ReviewInfor"].notnull()]
paid_games_withreviews.head()

11232


,AppID,AppName,Genre,Tags,Developer,Publisher,ReviewInfor,ReviewStatus,ReleaseDate,IsFree,Price
0,926170,Achromatic,Indie,"['Indie', 'Puzzle']",Studio Goya,Studio Goya,Positive(13)- 92% of the 13 user reviews for t...,True,"Sep 28, 2018",False,"{'currency': 'USD', 'initial': 499, 'final': 4..."
1,926180,Bubsy: Paws on Fire!,Action,"['Action', 'Adventure', 'Runner', 'Indie', 'Ps...",Choice Provisions,Accolade,Mostly Positive(28)- 71% of the 28 user review...,True,"May 16, 2019",False,"{'currency': 'USD', 'initial': 2499, 'final': ..."
2,926210,Incoming Evil,Indie,"['Indie', 'Action', 'First-Person', 'Horror', ...",Allyi Games,Allyi Games,9 user reviews- Need more user reviews to gene...,True,"Oct 5, 2019",False,"{'currency': 'USD', 'initial': 199, 'final': 7..."
4,926240,PsyBurst,Indie,"['Indie', 'Action', 'Fighting', 'Side', 'Scrol...",Hakagame,Hakagame,2 user reviews- Need more user reviews to gene...,True,"Sep 5, 2018",False,"{'currency': 'USD', 'initial': 299, 'final': 2..."
5,926250,Mirror Angel's Paradise,Indie,"['Indie', 'Action', 'Bullet', 'Hell']",Henez,Henez,7 user reviews- Need more user reviews to gene...,True,"Sep 11, 2018",False,"{'currency': 'USD', 'initial': 999, 'final': 9..."


In [22]:
# remove games which ReviewInfor contain "Need more user reviews"
paid_games_withmorereviews = paid_games_withreviews[~paid_games_withreviews["ReviewInfor"]\
                                                    .str.contains(r'.(Need more user reviews).*', na=False)]

print(len(paid_games_withmorereviews))
paid_games_withmorereviews.head()

23642


,AppID,AppName,Genre,Tags,Developer,Publisher,ReviewInfor,ReviewStatus,ReleaseDate,IsFree,Price
0,926170,Achromatic,Indie,"['Indie', 'Puzzle']",Studio Goya,Studio Goya,Positive(13)- 92% of the 13 user reviews for t...,True,"Sep 28, 2018",False,"{'currency': 'USD', 'initial': 499, 'final': 4..."
1,926180,Bubsy: Paws on Fire!,Action,"['Action', 'Adventure', 'Runner', 'Indie', 'Ps...",Choice Provisions,Accolade,Mostly Positive(28)- 71% of the 28 user review...,True,"May 16, 2019",False,"{'currency': 'USD', 'initial': 2499, 'final': ..."
8,926390,Infected Shelter,Action,"['Action', 'Roguelike', 'Indie', 'Action', 'Ad...",Dark Blue Games,Dark Blue Games,Very Positive(21)- 85% of the 21 user reviews ...,True,"Nov 11, 2019",False,"{'currency': 'USD', 'initial': 999, 'final': 9..."
14,926470,Kolkhoz: The Red Wedge,Adventure,"['Adventure', 'Indie', 'Story', 'Rich', '2D', ...",Deniz Kayacan,Deniz Kayacan,Positive(11)- 81% of the 11 user reviews for t...,True,"Oct 19, 2018",False,"{'currency': 'USD', 'initial': 999, 'final': 9..."
23,926650,Order of Battle: Endsieg,Strategy,"['Strategy', 'Simulation', 'Free', 'to', 'Play']",The Artistocrats,Slitherine Ltd.,Mostly Positive(18)- 72% of the 18 user review...,True,"Nov 13, 2018",False,"{'currency': 'USD', 'initial': 1499, 'final': ..."


In [32]:
# remove games with uncleaned ReleaseDate format: the normal date format length should be 12
print(len(paid_games_withmorereviews[paid_games_withmorereviews["ReleaseDate"].str.len()==12]))

df_clean_games = paid_games_withmorereviews[paid_games_withmorereviews["ReleaseDate"].str.len()==12]

16497


In [55]:
# add month and year features
#df_clean_games.ix[17073, 'ReleaseDate'] = "Dec 14, 2017"
df_clean_games["month"] = df_clean_games.ReleaseDate.apply(lambda x: x[0:3])
df_clean_games["year"] = df_clean_games.ReleaseDate.apply(lambda x: x[8:12])
df_clean_games.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,AppID,AppName,Genre,Tags,Developer,Publisher,ReviewInfor,ReviewStatus,ReleaseDate,IsFree,Price,month,year
0,926170,Achromatic,Indie,"['Indie', 'Puzzle']",Studio Goya,Studio Goya,Positive(13)- 92% of the 13 user reviews for t...,True,"Sep 28, 2018",False,"{'currency': 'USD', 'initial': 499, 'final': 4...",Sep,2018
1,926180,Bubsy: Paws on Fire!,Action,"['Action', 'Adventure', 'Runner', 'Indie', 'Ps...",Choice Provisions,Accolade,Mostly Positive(28)- 71% of the 28 user review...,True,"May 16, 2019",False,"{'currency': 'USD', 'initial': 2499, 'final': ...",May,2019
8,926390,Infected Shelter,Action,"['Action', 'Roguelike', 'Indie', 'Action', 'Ad...",Dark Blue Games,Dark Blue Games,Very Positive(21)- 85% of the 21 user reviews ...,True,"Nov 11, 2019",False,"{'currency': 'USD', 'initial': 999, 'final': 9...",Nov,2019
14,926470,Kolkhoz: The Red Wedge,Adventure,"['Adventure', 'Indie', 'Story', 'Rich', '2D', ...",Deniz Kayacan,Deniz Kayacan,Positive(11)- 81% of the 11 user reviews for t...,True,"Oct 19, 2018",False,"{'currency': 'USD', 'initial': 999, 'final': 9...",Oct,2018
23,926650,Order of Battle: Endsieg,Strategy,"['Strategy', 'Simulation', 'Free', 'to', 'Play']",The Artistocrats,Slitherine Ltd.,Mostly Positive(18)- 72% of the 18 user review...,True,"Nov 13, 2018",False,"{'currency': 'USD', 'initial': 1499, 'final': ...",Nov,2018


In [56]:
df_clean_games.year.unique()

array(['2018', '2019', '2017', '2002', '2016', '2020', '1999', '2011',
       '2009', '2015', '2001', '2014', '2010', '2004', '2000', '1998',
       '2003', '2008', '2013', '1995', '2012', '2006', '1997', '2007',
       '2005', '1994', '1991', '1990', '1993', '1996', '1984', '1983',
       '1989', '1992', '1988'], dtype=object)

In [58]:
df_clean_games_reasonableyear = df_clean_games[~df_clean_games.year.isin(['1994', '1991', '1990', '1993','1999', 
                                '1996', '1984', '1983', '1989', '1992', '1988', '1997', '1998', '1995'])]
print(df_clean_games_reasonableyear.year.unique())
print(len(df_clean_games_reasonableyear))

['2018' '2019' '2017' '2002' '2016' '2020' '2011' '2009' '2015' '2001'
 '2014' '2010' '2004' '2000' '2003' '2008' '2013' '2012' '2006' '2007'
 '2005']
16354


In [59]:
## filters: 
# paid (IsFree=False)
# ReviewInfor (not null, enough reviews)
# ReleaseDate is well formatted (12 digits)
# year: after 2000 before 2020

df_clean_games_reasonableyear.to_csv("df_clean_games_reasonableyear.csv", index=False)

In [3]:
df_games = pd.read_csv("df_clean_games_reasonableyear.csv")
df_games.head()

,AppID,AppName,Genre,Tags,Developer,Publisher,ReviewInfor,ReviewStatus,ReleaseDate,IsFree,Price,month,year
0,926170,Achromatic,Indie,"['Indie', 'Puzzle']",Studio Goya,Studio Goya,Positive(13)- 92% of the 13 user reviews for t...,True,"Sep 28, 2018",False,"{'currency': 'USD', 'initial': 499, 'final': 4...",Sep,2018
1,926180,Bubsy: Paws on Fire!,Action,"['Action', 'Adventure', 'Runner', 'Indie', 'Ps...",Choice Provisions,Accolade,Mostly Positive(28)- 71% of the 28 user review...,True,"May 16, 2019",False,"{'currency': 'USD', 'initial': 2499, 'final': ...",May,2019
2,926390,Infected Shelter,Action,"['Action', 'Roguelike', 'Indie', 'Action', 'Ad...",Dark Blue Games,Dark Blue Games,Very Positive(21)- 85% of the 21 user reviews ...,True,"Nov 11, 2019",False,"{'currency': 'USD', 'initial': 999, 'final': 9...",Nov,2019
3,926470,Kolkhoz: The Red Wedge,Adventure,"['Adventure', 'Indie', 'Story', 'Rich', '2D', ...",Deniz Kayacan,Deniz Kayacan,Positive(11)- 81% of the 11 user reviews for t...,True,"Oct 19, 2018",False,"{'currency': 'USD', 'initial': 999, 'final': 9...",Oct,2018
4,926650,Order of Battle: Endsieg,Strategy,"['Strategy', 'Simulation', 'Free', 'to', 'Play']",The Artistocrats,Slitherine Ltd.,Mostly Positive(18)- 72% of the 18 user review...,True,"Nov 13, 2018",False,"{'currency': 'USD', 'initial': 1499, 'final': ...",Nov,2018


In [5]:
df_games.ReviewInfor.isna().sum()

0

In [4]:
df_games["ReviewSentiment"] = df_games.ReviewInfor.apply(lambda x: x[0:x.index("(")])
df_games.head()

,AppID,AppName,Genre,Tags,Developer,Publisher,ReviewInfor,ReviewStatus,ReleaseDate,IsFree,Price,month,year,ReviewSentiment
0,926170,Achromatic,Indie,"['Indie', 'Puzzle']",Studio Goya,Studio Goya,Positive(13)- 92% of the 13 user reviews for t...,True,"Sep 28, 2018",False,"{'currency': 'USD', 'initial': 499, 'final': 4...",Sep,2018,Positive
1,926180,Bubsy: Paws on Fire!,Action,"['Action', 'Adventure', 'Runner', 'Indie', 'Ps...",Choice Provisions,Accolade,Mostly Positive(28)- 71% of the 28 user review...,True,"May 16, 2019",False,"{'currency': 'USD', 'initial': 2499, 'final': ...",May,2019,Mostly Positive
2,926390,Infected Shelter,Action,"['Action', 'Roguelike', 'Indie', 'Action', 'Ad...",Dark Blue Games,Dark Blue Games,Very Positive(21)- 85% of the 21 user reviews ...,True,"Nov 11, 2019",False,"{'currency': 'USD', 'initial': 999, 'final': 9...",Nov,2019,Very Positive
3,926470,Kolkhoz: The Red Wedge,Adventure,"['Adventure', 'Indie', 'Story', 'Rich', '2D', ...",Deniz Kayacan,Deniz Kayacan,Positive(11)- 81% of the 11 user reviews for t...,True,"Oct 19, 2018",False,"{'currency': 'USD', 'initial': 999, 'final': 9...",Oct,2018,Positive
4,926650,Order of Battle: Endsieg,Strategy,"['Strategy', 'Simulation', 'Free', 'to', 'Play']",The Artistocrats,Slitherine Ltd.,Mostly Positive(18)- 72% of the 18 user review...,True,"Nov 13, 2018",False,"{'currency': 'USD', 'initial': 1499, 'final': ...",Nov,2018,Mostly Positive


In [4]:
df_games["ReviewSentiment"].value_counts()

Positive                   4654
Mixed                      4057
Very Positive              4009
Mostly Positive            2554
Mostly Negative             746
Overwhelmingly Positive     175
Negative                    135
Very Negative                20
Overwhelmingly Negative       4
Name: ReviewSentiment, dtype: int64

In [5]:
df_games["Genre"].value_counts().head(20)

Action        2846
Indie         2464
Adventure     1631
Strategy      1457
Anime         1349
Casual        1337
Simulation    1336
RPG            749
Episodic       308
Racing         281
Sports         184
Sexual         130
Free           100
Early           99
Utilities       95
Visual          85
Nudity          80
Animation       78
Puzzle          72
Design          68
Name: Genre, dtype: int64

### Indie genre 

In [6]:
#df_games["Tags"] = df_games["Tags"].apply(lambda x: ast.literal_eval(x))

sub_indie=df_games[df_games.Genre=="Indie"]
sub_indie = sub_indie[sub_indie["Tags"].apply(lambda x: len(x)>=2)]
sub_indie["NewGenre"] = sub_indie["Tags"].apply(lambda x: x[1])

sub_others=df_games[df_games.Genre!="Indie"]
sub_others["NewGenre"] = sub_others["Genre"]

df_cleaned_games = pd.concat([sub_indie, sub_others], axis = 0)
df_cleaned_games = df_cleaned_games.reset_index(drop=True)
len(df_cleaned_games)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


16334

In [9]:
def clean_genres(data):
    genres = data.copy()
    # genre cleaning
    genres.loc[data['NewGenre'] == 'Massively', 'NewGenre'] = 'Multiplayer'

    genres.loc[data['NewGenre'] == 'RPGMaker', 'NewGenre'] = 'RPG'

    genres.loc[data['NewGenre'] == 'Zombies', 'NewGenre'] = 'Battle'
    genres.loc[data['NewGenre'] == 'Survival', 'NewGenre'] = 'Battle'
    genres.loc[data['NewGenre'] == 'PvP', 'NewGenre'] = 'Battle'
    genres.loc[data['NewGenre'] == 'Violent', 'NewGenre'] = 'Battle'

    genres.loc[data['NewGenre'] == 'Bullet', 'NewGenre'] = 'Shoot'

    genres.loc[data['NewGenre'] == 'Soccer', 'NewGenre'] = 'Sports'
    genres.loc[data['NewGenre'] == 'Swordplay', 'NewGenre'] = 'Sports'
    genres.loc[data['NewGenre'] == 'Football', 'NewGenre'] = 'Sports'
    genres.loc[data['NewGenre'] == 'Tennis', 'NewGenre'] = 'Sports'
    genres.loc[data['NewGenre'] == 'Motorbike', 'NewGenre'] = 'Sports'
    genres.loc[data['NewGenre'] == 'Baseball', 'NewGenre'] = 'Sports'
    genres.loc[data['NewGenre'] == 'Basketball', 'NewGenre'] = 'Sports'
    genres.loc[data['NewGenre'] == 'Golf', 'NewGenre'] = 'Sports'
    genres.loc[data['NewGenre'] == 'Boxing', 'NewGenre'] = 'Sports'
    genres.loc[data['NewGenre'] == 'Fishing', 'NewGenre'] = 'Sports'
    genres.loc[data['NewGenre'] == 'Bikes', 'NewGenre'] = 'Sports'

    genres.loc[data['NewGenre'] == 'Action', 'NewGenre'] = 'Action-Adventure'
    genres.loc[data['NewGenre'] == 'Adventure', 'NewGenre'] = 'Action-Adventure'
    return genres

def filter_top_genres(data):
    top_genres_list = ['Multiplayer', 'RPG', 'Battle', 'Shoot', 'Sports', 'Action-Adventure', \
              'Sandbox', 'Simulation', 'Fighting', 'Racing']
    # filter the top 10 genres
    top_genres = data.loc[data['NewGenre'].isin(top_genres_list) == True,]
    return top_genres

In [11]:
df_top_genres = clean_genres(df_cleaned_games)
df_games_topgenres = filter_top_genres(df_top_genres)
df_games_topgenres["NewGenre"].value_counts()

Action-Adventure    5452
Simulation          1498
RPG                  858
Racing               299
Sports               226
Battle               130
Multiplayer           91
Shoot                 45
Fighting              23
Sandbox               12
Name: NewGenre, dtype: int64

In [12]:
df_top3genre = df_games_topgenres[df_games_topgenres["NewGenre"].isin(['Action-Adventure', 'Simulation','RPG'])]
len(df_top3genre)

7808

In [13]:
df_top3genre.to_csv("df_top3genre.csv", index=False)

## collect reviews

In [ ]:
# # https://github.com/woctezuma/download-steam-reviews
# # https://store.steampowered.com/app/618980/Wanderland_Advanced_Adventurer_Pack/
# # no review: https://store.steampowered.com/app/619610/Ultratank/
# ids = ["523660", '618980']

# request_params = dict()
# request_params['language'] = 'english'
# # Reference: https://partner.steamgames.com/doc/store/getreviews

# game_reviews = steamreviews.download_reviews_for_app_id_batch(ids, chosen_request_params=request_params)
# game_reviews

In [4]:
df_top3genre = pd.read_csv("df_top3genre.csv")
ids = df_top3genre.AppID.values
# ids = list(set(ids) - set(x))
request_params = dict()
request_params['language'] = 'english'
game_reviews = steamreviews.download_reviews_for_app_id_batch(ids, chosen_request_params=request_params)

Loading idprocessed_on_20200531.txt
Creating idprocessed_on_20200531.txt
[appID = 925340] expected #reviews = 23
[appID = 925340] num_reviews = 23 (expected: 23)
[appID = 925440] expected #reviews = 33
[appID = 925440] num_reviews = 33 (expected: 33)
[appID = 925520] expected #reviews = 54
[appID = 925520] num_reviews = 54 (expected: 54)
[appID = 922520] expected #reviews = 1
[appID = 922520] num_reviews = 1 (expected: 1)
[appID = 921770] expected #reviews = 13
[appID = 921770] num_reviews = 13 (expected: 13)
[appID = 918380] expected #reviews = 101
[appID = 918380] num_reviews = 101 (expected: 101)
[appID = 916730] expected #reviews = 860
[appID = 916730] num_reviews = 860 (expected: 860)
[appID = 913550] expected #reviews = 1
[appID = 913550] num_reviews = 1 (expected: 1)
[appID = 912660] expected #reviews = 13
[appID = 912660] num_reviews = 13 (expected: 13)
[appID = 907890] expected #reviews = 16
[appID = 907890] num_reviews = 16 (expected: 16)
[appID = 905340] expected #reviews = 

[appID = 982590] expected #reviews = 33
[appID = 982590] num_reviews = 33 (expected: 33)
[appID = 982110] expected #reviews = 29
[appID = 982110] num_reviews = 29 (expected: 29)
[appID = 981420] expected #reviews = 50
[appID = 981420] num_reviews = 50 (expected: 50)
[appID = 980590] expected #reviews = 16
[appID = 980590] num_reviews = 16 (expected: 16)
[appID = 979530] expected #reviews = 10
[appID = 979530] num_reviews = 10 (expected: 10)
[appID = 977570] expected #reviews = 65
[appID = 977570] num_reviews = 65 (expected: 65)
[appID = 976120] expected #reviews = 46
[appID = 976120] num_reviews = 46 (expected: 46)
[appID = 973910] expected #reviews = 31
[appID = 973910] num_reviews = 31 (expected: 31)
[appID = 971650] expected #reviews = 30
[appID = 971650] num_reviews = 30 (expected: 30)
[appID = 970640] expected #reviews = 14
[appID = 970640] num_reviews = 14 (expected: 14)
[appID = 961860] expected #reviews = 16
[appID = 961860] num_reviews = 16 (expected: 16)
[appID = 959030] expe

[appID = 642600] expected #reviews = 13
[appID = 642600] num_reviews = 13 (expected: 13)
[appID = 641950] expected #reviews = 14
[appID = 641950] num_reviews = 14 (expected: 14)
[appID = 640380] expected #reviews = 135
[appID = 640380] num_reviews = 135 (expected: 135)
[appID = 640120] expected #reviews = 73
[appID = 640120] num_reviews = 73 (expected: 73)
[appID = 636950] expected #reviews = 10
[appID = 636950] num_reviews = 10 (expected: 10)
[appID = 636150] expected #reviews = 183
[appID = 636150] num_reviews = 183 (expected: 183)
[appID = 630360] expected #reviews = 14
[appID = 630360] num_reviews = 14 (expected: 14)
[appID = 629840] expected #reviews = 281
[appID = 629840] num_reviews = 281 (expected: 281)
[appID = 628800] expected #reviews = 72
[appID = 628800] num_reviews = 72 (expected: 72)
[appID = 625740] expected #reviews = 80
[appID = 625740] num_reviews = 80 (expected: 80)
[appID = 625820] expected #reviews = 20
[appID = 625820] num_reviews = 20 (expected: 20)
[appID = 624

[appID = 768460] expected #reviews = 26
[appID = 768460] num_reviews = 26 (expected: 26)
[appID = 766630] expected #reviews = 54
[appID = 766630] num_reviews = 54 (expected: 54)
[appID = 763050] expected #reviews = 13
[appID = 763050] num_reviews = 13 (expected: 13)
[appID = 763070] expected #reviews = 17
[appID = 763070] num_reviews = 17 (expected: 17)
[appID = 762080] expected #reviews = 44
[appID = 762080] num_reviews = 44 (expected: 44)
[appID = 758660] expected #reviews = 32
[appID = 758660] num_reviews = 32 (expected: 32)
[appID = 757870] expected #reviews = 11
[appID = 757870] num_reviews = 11 (expected: 11)
[appID = 757420] expected #reviews = 7
[appID = 757420] num_reviews = 7 (expected: 7)
[appID = 757580] expected #reviews = 17
[appID = 757580] num_reviews = 17 (expected: 17)
[appID = 757080] expected #reviews = 12
[appID = 757080] num_reviews = 12 (expected: 12)
[appID = 755670] expected #reviews = 25
[appID = 755670] num_reviews = 25 (expected: 25)
[appID = 752550] expecte

[appID = 1054790] expected #reviews = 12
[appID = 1054790] num_reviews = 12 (expected: 12)
[appID = 1053730] expected #reviews = 12
[appID = 1053730] num_reviews = 12 (expected: 12)
[appID = 1048300] expected #reviews = 15
[appID = 1048300] num_reviews = 15 (expected: 15)
[appID = 1044020] expected #reviews = 17
[appID = 1044020] num_reviews = 17 (expected: 17)
[appID = 1043230] expected #reviews = 30
[appID = 1043230] num_reviews = 30 (expected: 30)
[appID = 1038100] expected #reviews = 9
[appID = 1038100] num_reviews = 9 (expected: 9)
[appID = 1036870] expected #reviews = 3
[appID = 1036870] num_reviews = 3 (expected: 3)
[appID = 1035020] expected #reviews = 7
[appID = 1035020] num_reviews = 7 (expected: 7)
[appID = 1033510] expected #reviews = 5
[appID = 1033510] num_reviews = 5 (expected: 5)
[appID = 1029980] expected #reviews = 75
[appID = 1029980] num_reviews = 75 (expected: 75)
[appID = 1029080] expected #reviews = 11
[appID = 1029080] num_reviews = 11 (expected: 11)
[appID = 10

...

[appID = 762520] expected #reviews = 33
[appID = 762520] num_reviews = 33 (expected: 33)
[appID = 838630] expected #reviews = 293
[appID = 838630] num_reviews = 293 (expected: 293)
[appID = 952890] expected #reviews = 103
[appID = 952890] num_reviews = 103 (expected: 103)
[appID = 1058020] expected #reviews = 1284
[appID = 1058020] num_reviews = 1284 (expected: 1284)
[appID = 1122100] expected #reviews = 32
[appID = 1122100] num_reviews = 32 (expected: 32)
[appID = 257350] expected #reviews = 2881
[appID = 257350] num_reviews = 2881 (expected: 2881)
[appID = 359900] expected #reviews = 390
[appID = 359900] num_reviews = 390 (expected: 390)
[appID = 252430] expected #reviews = 115
[appID = 252430] num_reviews = 115 (expected: 115)
[appID = 1004770] expected #reviews = 194
[appID = 1004770] num_reviews = 194 (expected: 194)
[appID = 228360] expected #reviews = 873
[appID = 228360] num_reviews = 873 (expected: 873)
[appID = 408960] expected #reviews = 137
[appID = 408960] num_reviews = 13

[appID = 950050] expected #reviews = 50
[appID = 950050] num_reviews = 50 (expected: 50)
[appID = 957780] expected #reviews = 12
[appID = 957780] num_reviews = 12 (expected: 12)
[appID = 279720] expected #reviews = 228
[appID = 279720] num_reviews = 228 (expected: 228)
[appID = 481110] expected #reviews = 405
[appID = 481110] num_reviews = 405 (expected: 405)
[appID = 526510] expected #reviews = 25
[appID = 526510] num_reviews = 25 (expected: 25)
[appID = 10270] expected #reviews = 269
[appID = 10270] num_reviews = 269 (expected: 269)
[appID = 1061300] expected #reviews = 12
[appID = 1061300] num_reviews = 12 (expected: 12)
[appID = 591790] expected #reviews = 148
[appID = 591790] num_reviews = 148 (expected: 148)
[appID = 1206570] expected #reviews = 3
[appID = 1206570] num_reviews = 3 (expected: 3)
[appID = 551050] expected #reviews = 18
[appID = 551050] num_reviews = 18 (expected: 18)
[appID = 1166290] expected #reviews = 722
[appID = 1166290] num_reviews = 722 (expected: 722)
[appI

[appID = 1007400] expected #reviews = 64
[appID = 1007400] num_reviews = 64 (expected: 64)
[appID = 340310] expected #reviews = 291
[appID = 340310] num_reviews = 291 (expected: 291)
[appID = 581310] expected #reviews = 23
[appID = 581310] num_reviews = 23 (expected: 23)
[appID = 834280] expected #reviews = 247
[appID = 834280] num_reviews = 247 (expected: 247)
[appID = 1087424] expected #reviews = 9
[appID = 1087424] num_reviews = 9 (expected: 9)
[appID = 322500] expected #reviews = 14739
Number of queries 150 reached. Cooldown: 310 seconds
[appID = 322500] num_reviews = 14739 (expected: 14739)
[appID = 23380] expected #reviews = 184
[appID = 23380] num_reviews = 184 (expected: 184)
[appID = 250680] expected #reviews = 487
[appID = 250680] num_reviews = 487 (expected: 487)
[appID = 444160] expected #reviews = 82
[appID = 444160] num_reviews = 82 (expected: 82)
[appID = 539090] expected #reviews = 19
[appID = 539090] num_reviews = 19 (expected: 19)
[appID = 1036260] expected #reviews =

[appID = 1170120] expected #reviews = 26
[appID = 1170120] num_reviews = 26 (expected: 26)
[appID = 722540] expected #reviews = 11
[appID = 722540] num_reviews = 11 (expected: 11)
[appID = 1185740] expected #reviews = 7
[appID = 1185740] num_reviews = 7 (expected: 7)
[appID = 807790] expected #reviews = 15
[appID = 807790] num_reviews = 15 (expected: 15)
[appID = 1252870] expected #reviews = 70
[appID = 1252870] num_reviews = 70 (expected: 70)
[appID = 488550] expected #reviews = 323
[appID = 488550] num_reviews = 323 (expected: 323)
[appID = 524640] expected #reviews = 185
Number of queries 150 reached. Cooldown: 310 seconds
[appID = 524640] num_reviews = 185 (expected: 185)
[appID = 1112790] expected #reviews = 1052
[appID = 1112790] num_reviews = 1052 (expected: 1052)
[appID = 330390] expected #reviews = 953
[appID = 330390] num_reviews = 953 (expected: 953)
[appID = 283620] expected #reviews = 315
[appID = 283620] num_reviews = 315 (expected: 315)
[appID = 353070] expected #reviews

[appID = 959520] expected #reviews = 238
[appID = 959520] num_reviews = 238 (expected: 238)
[appID = 523640] expected #reviews = 99
[appID = 523640] num_reviews = 99 (expected: 99)
[appID = 595300] expected #reviews = 18
[appID = 595300] num_reviews = 18 (expected: 18)
[appID = 763030] expected #reviews = 59
[appID = 763030] num_reviews = 59 (expected: 59)
[appID = 1074220] expected #reviews = 12
[appID = 1074220] num_reviews = 12 (expected: 12)
[appID = 1138970] expected #reviews = 14
[appID = 1138970] num_reviews = 14 (expected: 14)
[appID = 563390] expected #reviews = 13
[appID = 563390] num_reviews = 13 (expected: 13)
[appID = 1019450] expected #reviews = 37
[appID = 1019450] num_reviews = 37 (expected: 37)
[appID = 710490] expected #reviews = 87
[appID = 710490] num_reviews = 87 (expected: 87)
[appID = 805660] expected #reviews = 50
[appID = 805660] num_reviews = 50 (expected: 50)
[appID = 740550] expected #reviews = 48
[appID = 740550] num_reviews = 48 (expected: 48)
[appID = 837

In [5]:
import datetime

GameID = []
SteamID = []
ReviewID = []
NumGamesOwned = []
NumReviewsGiven = []
PlayTime = []
PlayTimeLastTwoWeeks = []
LastPlayTime = []
Language = []
Review = []
TimeCreated = []
Recommended = []
VotesUp = []
VoteFunny = []
WeightedVoteScore = []
CommentCount = []
SteamPurchase = []
WriteEarly = []


#ids = ["523660", '618980', '667290']
#ids=x+y+z+a

for i in range(len(ids)):
    gameid = ids[i]
    with open(f'data/review_{gameid}.json', 'r') as f:
        review_dict = json.load(f)
    
    if len(review_dict["reviews"]) != 0:
        for j in range(len(review_dict["reviews"])):
            try: 
                reviewids = list(review_dict["reviews"])
                doc = review_dict["reviews"][reviewids[j]]
                author_profile = doc["author"]
                language = doc["language"]
                steamid = author_profile["steamid"]
                num_games_owned = author_profile["num_games_owned"]
                num_reviews = author_profile["num_reviews"]
                playtime_forever = author_profile["playtime_forever"]
                playtime_last_two_weeks = author_profile["playtime_last_two_weeks"]
                last_played = datetime.datetime.fromtimestamp(author_profile["last_played"])
                review = doc["review"]
                timestamp_created = datetime.datetime.fromtimestamp(doc["timestamp_created"])
                recommended = doc["voted_up"]
                votes_up = doc["votes_up"]
                votes_funny = doc["votes_funny"]
                weighted_vote_score = doc["weighted_vote_score"]
                comment_count = doc["comment_count"]
                steam_purchase = doc["steam_purchase"]
                write_early = doc["written_during_early_access"]
                
            except TypeError: 
                continue
            
            GameID.append(gameid)
            SteamID.append(steamid)
            ReviewID.append(reviewids[j])
            NumGamesOwned.append(num_games_owned)
            NumReviewsGiven.append(num_reviews)
            PlayTime.append(playtime_forever)
            PlayTimeLastTwoWeeks.append(playtime_last_two_weeks)
            LastPlayTime.append(last_played)
            Language.append(language)
            Review.append(review)
            TimeCreated.append(timestamp_created)
            Recommended.append(recommended)
            VotesUp.append(votes_up)
            VoteFunny.append(votes_funny)
            WeightedVoteScore.append(weighted_vote_score)
            CommentCount.append(comment_count)
            SteamPurchase.append(steam_purchase)
            WriteEarly.append(write_early)

In [6]:
df_reviews = pd.DataFrame(list(zip(GameID, SteamID, ReviewID, NumGamesOwned, NumReviewsGiven, PlayTime, 
                PlayTimeLastTwoWeeks, LastPlayTime, Language, Review, TimeCreated, Recommended, VotesUp, 
                VoteFunny, WeightedVoteScore, CommentCount, SteamPurchase, WriteEarly)),
                columns=["GameID", "SteamID", "ReviewID", "NumGamesOwned", "NumReviewsGiven", "PlayTime", 
                         "PlayTimeLastTwoWeeks","LastPlayTime", "Language", "Review", "TimeCreated", 
                         "Recommended", "VotesUp", "VoteFunny", "WeightedVoteScore", "CommentCount", 
                         "SteamPurchase", "WriteEarly"])
df_reviews.head(5)

,GameID,SteamID,ReviewID,NumGamesOwned,NumReviewsGiven,PlayTime,PlayTimeLastTwoWeeks,LastPlayTime,Language,Review,TimeCreated,Recommended,VotesUp,VoteFunny,WeightedVoteScore,CommentCount,SteamPurchase,WriteEarly
0,925340,76561198200016459,62850082,307,13,192,0,2020-02-02 03:00:43,english,Yuu is a student attending university in Tokyo...,2020-02-02 03:03:18,True,7,0,0.607030391693115234,0,False,False
1,925340,76561198080360383,60947171,346,27,151,0,2019-12-30 14:48:34,english,"A short, but (bitter)sweet game. Really makes ...",2019-12-30 14:48:24,True,2,0,0.546827793121337891,0,True,False
2,925340,76561198412449978,54724410,293,288,73,0,2019-08-20 16:12:03,english,Welll this VN was Something Interesting How wo...,2019-08-20 16:28:09,True,5,1,0.587897598743438721,0,True,False
3,925340,76561198130402546,54220873,322,38,162,1,2020-05-21 18:52:57,english,"A VN with a short, sweet story that focuses on...",2019-07-21 18:43:37,True,1,0,0.523809552192687988,0,True,False
4,925340,76561198136886006,50769360,453,28,170,0,2019-05-22 11:04:59,english,A young man from rural Japan comes home from u...,2019-05-22 11:10:55,True,0,0,0,0,True,False


In [7]:
len(df_reviews)

1819058

In [8]:
df_reviews.to_csv("df_reviews.csv", index=False)

In [42]:
review_dict["reviews"][list(review_dict["reviews"])[0]]

{'recommendationid': '68187610',
 'author': {'steamid': '76561198045740795',
  'num_games_owned': 26,
  'num_reviews': 1,
  'playtime_forever': 232,
  'playtime_last_two_weeks': 232,
  'last_played': 1588097810},
 'language': 'english',
 'review': 'Tremendous, exhilarating fun!\nThe attention to detail in this game is second to none, every factor of gameplay and level design has been intentionally made to evoke the feelings of playing with your action figures as a kid.\nIt\'s incredibly refreshing to see a game in this day and age which gets you to play the game to unlock new characters and armour pieces rather than relying on microtransactions up the ying yang. Extra points for this.\nThe variety in enemy types, weapon pickups and character designs took a lot of evident effort. I believe the creators should be truly commended for this. The first time bouncy balls showed up had me go "Oh S**t there\'s goddamn bouncy balls!" out loud... I never thought I\'d be afraid of bouncy balls!\n\